In [ ]:
#!pip install numpy pandas matplotlib seaborn statsmodels scipy scikit-learn

#import relevant libraries
import os
import sys

import numpy as np
import pandas as pd
import sys; sys.path
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests as fdr
from matplotlib import colors
from scipy import stats

from sklearn.metrics import explained_variance_score, r2_score, classification_report
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, GroupKFold, GroupShuffleSplit, StratifiedKFold
from sklearn.svm import SVC
from scipy import stats
from sklearn.utils import shuffle
from datetime import datetime


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Create a DataFrame with rsfc predictor data
ABCD_base_dir = 'PATH_TO_DIR'
ABCD_results_dir = 'PATH_TO_DIR'

data = pd.read_csv(os.path.join(ABCD_base_dir, 'females_predictors.csv'), header=None)
fc_f = pd.DataFrame(data)

# Access the values as a numpy array
values = fc_f.values

In [ ]:
# Create a DataFrame with labels
data = pd.read_csv(os.path.join(ABCD_base_dir, 'females_labels.csv'), header=None)
gender_p_f_sum = pd.DataFrame(data)

# Access the values as a numpy array
T = gender_p_f_sum.T

In [ ]:
# Create a DataFrame with site data
data = pd.read_csv(os.path.join(ABCD_base_dir, 'females_site.csv'), header=None)
site_f = pd.DataFrame(data)

# Access the values as a numpy array
values = site_f.values

In [ ]:
#set up predictive models
#number of repetitions you want to perform (100 for 'true', 1000 for 'null')
rep = 100

#number of folds you want in the cross-validation
k = 3
#proportion of data you want in your training set and test set
train_size = .80
test_size = 1-train_size

#regression model type

from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Create a pipeline with standardization and Ridge regression
regr = make_pipeline(StandardScaler(), Ridge(max_iter=1000000, solver='lsqr'))

#set hyperparameter grid space you want to search through for the model
#adapted from the Thomas Yeo Lab Github: 
#ThomasYeoLab/CBIG/blob/master/stable_projects/predict_phenotypes/He2019_KRDNN/KR_HCP/CBIG_KRDNN_KRR_HCP.m
#alphas = [0, 0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3,
#          3.5, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 150, 200, 300, 500, 700, 1000, 2000, 
#          3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

alphas = [0.001, 0.01, 0.1, 1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 100, 
          150, 200, 300, 500, 700, 1000]

#alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]

#param grid set to the hyperparamters you want to search through
paramGrid ={'ridge__alpha': alphas}

#set x data to be the input variable you want to use
X = fc_f.values
Y = np.ravel(gender_p_f_sum.T)
site = np.asarray(site_f.values).astype(int)
site = site.ravel()

#number of features 
n_feat = X.shape[1]

pred_name = 'youth_report'
pred_sex = 'f'


In [ ]:
#create variables to store relevant data/outputs
#r^2 - coefficient of determination
r2 = np.zeros([rep])
#explained variance
var = np.zeros([rep])
#correlation between true and predicted (aka prediction accuracy)
corr = np.zeros([rep])
#optimised alpha (hyperparameter)
opt_alpha = np.zeros([rep])
#predictions made by the model
#don't need to save any of these right now
#preds = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#true test values for cognition
#cogtest = np.zeros([rep,n_cog,int(np.ceil(X.shape[0]*test_size))])
#feature importance extracted from the model
featimp = np.zeros([rep,n_feat])
#for when the feat weights get haufe-inverted
#featimp_haufe = np.zeros([rep,n_feat])

In [ ]:
#iterate through the diff train/test splits

#alphas = np.load(ABCD_results_dir + '/fc_alpha_' + pred_name + '_opt.npy') ##only need this for the null ones

for p in range(rep):
    
    #print model # you're on
    print('Model %d' %(p+1))
    
    #Y_shuffle = shuffle(Y, random_state=p) ##only need this for the null ones
    
    #print time
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    
    #split into train/test data
    train_inds, test_inds = next(GroupShuffleSplit(test_size=1-train_size, n_splits=1, random_state = p).split(X, groups=site))
    
    #set x values based on indices from split
    x_train = X[train_inds]
    x_test = X[test_inds]
        
    #set y values based on indices from split  
    beh_train = Y[train_inds]
    beh_test = Y[test_inds]
    
    #set y values based on indices from split ##only need this for the null ones
    #beh_train = Y_shuffle[train_inds]
    #beh_test = Y_shuffle[test_inds]
    
    site_train = site[train_inds]
    site_test = site[test_inds] 
    
    #convert y values to to double
    y_train = np.double(beh_train)
    y_test = np.double(beh_test)



    #create variables to store nested CV scores, and best parameters from hyperparameter optimisation
    best_scores = []
    best_params = []
    

        
    #set parameters for inner and outer loops for CV
    cv_split = GroupKFold(n_splits=k)
        
    #print ("Optimising Models")
            
    #define regressor with grid-search CV for inner loop
    gridSearch = GridSearchCV(estimator=regr, param_grid=paramGrid, n_jobs=-1, verbose=0, cv=cv_split, scoring='explained_variance')
    
    #fit regressor to the model, use site ID as group category again
    gridSearch.fit(x_train, y_train, groups=site_train)

    #save parameters corresponding to the best score
    best_params.append(list(gridSearch.best_params_.values()))
    best_scores.append(gridSearch.best_score_)
        

    #print ("Evaluating Models")
        
    #save optimised alpha values
    opt_alpha[p] = best_params[best_scores.index(np.max(best_scores))][0]
    
    
    #rand_alpha = np.random.choice(alphas)
    
    #fit optimized models
    #model = Ridge(alpha = opt_alpha[p], normalize=True, max_iter=1000000, solver='lsqr')

    model = make_pipeline(StandardScaler(), Ridge(alpha = opt_alpha[p], max_iter=1000000, solver='lsqr'))

    #model = Ridge(alpha = rand_alpha, normalize=True, max_iter=1000000, solver='lsqr')

    model.fit(x_train, y_train);
        
        
    #evaluate model
    r2[p]=model.score(x_test,y_test)
    #print(r2[p])
        
    #generate predictions in test set
    preds = []
    preds = model.predict(x_test).ravel()
    
        
    #compute explained variance 
    var[p] = explained_variance_score(y_test, preds)
    #print(var[p])


    #compute correlation between true and predicted (prediction accuracy)
    corr[p] = np.corrcoef(y_test.ravel(), preds)[1,0]
    #print(corr[p])
    
    #print ("Haufe-Transforming Feature Weights")
    cov_x = []
    cov_y = []
    
    #extract feature importance
    featimp[p,:] = model.named_steps['ridge'].coef_
    #compute Haufe-inverted feature weights
    cov_x = np.cov(np.transpose(x_train))
    cov_y = np.cov(y_train)
    #featimp_haufe[p,:] = np.matmul(cov_x,featimp[p,:])*(1/cov_y)
                
    #save results
    np.save((ABCD_results_dir + '/fc_r2_' + pred_name + pred_sex + '.npy'),r2)
    np.save((ABCD_results_dir + '/fc_var_' + pred_name + pred_sex + '.npy'),var)
    np.save((ABCD_results_dir + '/fc_corr_' + pred_name + pred_sex + '.npy'),corr)
    #np.save((ABCD_results_dir + '/fc_alpha_' + pred_name + pred_sex + '.npy'),opt_alpha)
    #np.save((ABCD_results_dir + '/fc_featimp_' + pred_name + pred_sex + '.npy'),featimp)
    #np.save((ABCD_results_dir + '/fc_featimp_haufe_' + pred_name + pred_sex + '.npy'),featimp_haufe_f)

print("Finished")